In [1]:
import jax

In [2]:
jax.devices()

[CudaDevice(id=0)]

In [3]:
from flax import nnx
import optax
import jax.numpy as jnp


In [4]:
class Encoder(nnx.Module):
    """VAE Encoder"""
    def __init__(self, latent_dim: int, image_flatten_dim: int, internal_dim:int, rngs: nnx.Rngs):
        self.dense_layer = nnx.Linear(in_features=image_flatten_dim, out_features=internal_dim, rngs=rngs)
        self.mean_layer = nnx.Linear(in_features=internal_dim, out_features=latent_dim, rngs=rngs)
        self.logvar_layer = nnx.Linear(in_features=internal_dim, out_features=latent_dim, rngs=rngs)
    def __call__(self, x: jax.Array):
        x = self.dense_layer(x)
        x = jax.nn.relu(x)
        means = self.mean_layer(x)
        logvars = self.logvar_layer(x)
        return (means, logvars)
        

In [5]:
class Decoder(nnx.Module):
    """VAE Decoder"""
    def __init__(self, latent_dim: int, image_flatten_dim: int, internal_dim:int, rngs: nnx.Rngs):
        self.dense_layer1 = nnx.Linear(in_features=latent_dim, out_features=internal_dim, rngs=rngs)
        self.dense_layer2 = nnx.Linear(in_features=internal_dim, out_features=image_flatten_dim, rngs=rngs)
    def __call__(self, x: jax.Array):
        x = self.dense_layer1(x)
        x = jax.nn.relu(x)
        x = self.dense_layer2(x)
        return x

In [6]:
class VAE(nnx.Module):
    """VAE together"""
    def __init__(self, latent_dim:int, image_flatten_dim:int, internal_dim:int, rngs: nnx.Rngs):
        self.encoder = Encoder(latent_dim=latent_dim, image_flatten_dim=image_flatten_dim, internal_dim=internal_dim, rngs=rngs)
        self.decoder = Decoder(latent_dim=latent_dim, image_flatten_dim=image_flatten_dim, internal_dim=internal_dim, rngs=rngs)
        self.random_for_reparam = rngs.reparam()
    def __call__(self, x: jax.Array):
        mean, logvar = self.encoder(x)
        x = self.reparam(mean, logvar)
        x = self.decoder(x)
        return x, mean, logvar

    def reparam(self, mean: jax.Array, logvars: jax.Array):
        std = jnp.exp(logvars/2.0)
        eps = jax.random.normal(self.random_for_reparam, logvars.shape)
        return mean + eps*std
        

# Create Datasets

In [8]:
import datasets
from PIL import Image
from IPython.display import display

In [9]:
dataset = datasets.load_dataset("ylecun/mnist")

In [10]:
fl = dataset["train"]["image"][0]

In [11]:
jnp.array([jnp.array([1,2]), jnp.array([3,4])])

Array([[1, 2],
       [3, 4]], dtype=int32)

In [12]:
iter_data = dataset["train"].map(lambda x: {"x": jnp.array([jnp.array(i) for i in x["image"]])}, remove_columns=dataset["train"].column_names, batched=True, batch_size=32)

# Training

In [13]:
def kl_divergence(mean, logvar):
  return -0.5 * jnp.sum(1 + logvar - jnp.square(mean) - jnp.exp(logvar))

In [14]:
def reconstruct_loss(logits, labels):
    logits = jax.nn.log_sigmoid(logits)
    return -jnp.sum(
        labels * logits + (1.0 - labels) * jnp.log(-jnp.expm1(logits))
    )

In [56]:
model = VAE(10, 28*28, 256, nnx.Rngs(0))
tx = optax.adam(0.001)
optimizer = nnx.Optimizer(model, tx)

In [57]:
def loss_fn(model, data):
    logits, mean, logvar = model(data)
    divergence = kl_divergence(mean, logvar)
    reconstruct = reconstruct_loss(logits, data)
    return divergence + reconstruct


In [58]:
@nnx.jit
def train_step(model: VAE, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, data):
    grad_fn = nnx.value_and_grad(loss_fn)
    loss, grads = grad_fn(model, data)
    optimizer.update(grads)
    metrics.update(loss=loss)
    

In [59]:
metrics = nnx.MultiMetric(
  loss=nnx.metrics.Average('loss'),
)

In [ ]:
for _ in range(10):
    for step, batch in enumerate(iter_data.shuffle()['x']):
        data = jnp.array(jnp.array(batch)!=0, dtype=jnp.float32).reshape(-1, 28*28)
        train_step(model, optimizer, metrics, data)
        if step%100 == 0:
            print([(m, v) for m,v in metrics.compute().items()])

# Inference

In [61]:
test_data = dataset["test"]["image"]

In [117]:
test_data[1]

In [118]:
prediction = jax.nn.sigmoid(model(jnp.array(jnp.array(test_data[1]).reshape(-1)!=0, dtype=jnp.float32))[0]).reshape((28,28))
bytes=(255*(prediction>0.5).astype(dtype=jnp.int8)).tobytes()
Image.frombytes('L', data=bytes, size=(28,28))